## Accelerate Inference: Neural Network Pruning

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.layers import *

print(tf.version.VERSION)

2.9.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cp  "/content/drive/MyDrive/dataset.tar.gz" .

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# untar
!tar -xvzf dataset.tar.gz
# load train
train_images = pickle.load(open('train_images.pkl', 'rb'))
train_labels = pickle.load(open('train_labels.pkl', 'rb'))
# load val
val_images = pickle.load(open('val_images.pkl', 'rb'))
val_labels = pickle.load(open('val_labels.pkl', 'rb'))

train_images.pkl
train_labels.pkl
val_images.pkl
val_labels.pkl


In [ ]:
# Define the neural network architecture (don't change this)

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

In [ ]:
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 25, 25, 32)        896       
                                                                 
 activation_24 (Activation)  (None, 25, 25, 32)        0         
                                                                 
 conv2d_17 (Conv2D)          (None, 23, 23, 32)        9248      
                                                                 
 activation_25 (Activation)  (None, 23, 23, 32)        0         
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 11, 11, 32)       0         
 2D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 11, 11, 32)        0         
                                                      

In [ ]:
def sparsity(weights_val):
  size = 1
  n_zeros = np.count_nonzero(weights_val==0)
  # print(n_zeros)
  for dim in np.shape(weights_val):
    size = size * dim
  # print(size)
  sparsity = n_zeros/size
  return n_zeros, size, sparsity

In [ ]:
# you can use the default hyper-parameters for training, 
# and val accuracy ~59% after 25 epochs and > 63% after 50 epochs

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])



history = model.fit(train_images, train_labels, batch_size=16, epochs=50, 
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32 

Epoch 1/50


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1405/1405 [==============================] - 7s 4ms/step - loss: 1.4657 - accuracy: 0.3451 - val_loss: 1.3077 - val_accuracy: 0.4352
Epoch 2/50
1405/1405 [==============================] - 6s 4ms/step - loss: 1.2983 - accuracy: 0.4474 - val_loss: 1.2245 - val_accuracy: 0.4796
Epoch 3/50
1405/1405 [==============================] - 6s 4ms/step - loss: 1.2350 - accuracy: 0.4857 - val_loss: 1.1563 - val_accuracy: 0.5271
Epoch 4/50
1405/1405 [==============================] - 6s 4ms/step - loss: 1.1810 - accuracy: 0.5161 - val_loss: 1.1192 - val_accuracy: 0.5481
Epoch 5/50
1405/1405 [==============================] - 6s 4ms/step - loss: 1.1420 - accuracy: 0.5364 - val_loss: 1.0745 - val_accuracy: 0.5731
Epoch 6/50
1405/1405 [==============================] - 6s 4ms/step - loss: 1.1030 - accuracy: 0.5569 - val_loss: 1.0595 - val_accuracy: 0.5683
Epoch 7/50
1405/1405 [==============================] - 6s 4ms/step - loss: 1.0677 - accuracy: 0.5747 - val_loss: 1.0305 - val_accuracy: 0.5834
Epo

In [ ]:
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 4ms/step - loss: 0.7009 - accuracy: 0.7358


In [ ]:
original_weights = model.get_weights()

### Retrain Attempt
## Score: 0.8467



In [ ]:
model.set_weights(original_weights)

In [ ]:
for i in range (300):
  if i < 100:
    thresh = 0.095
  elif i > 99 and i < 150:
    thresh = 0.1
  elif i > 149 and i < 200:
    thresh = 0.13
  else:
    thresh = 0.15

  print("Epoch: {}".format(i + 1))

  history = model.fit(train_images, train_labels, batch_size=16, epochs= 1, 
                    validation_data=(val_images, val_labels))
  weights = model.get_weights()

  new_weights = np.array(weights)
  for val in range (12):
    new_weights[val][np.where(np.abs(new_weights[val]) < thresh)] = 0.0 
  model.set_weights(new_weights)
  final_sparsity = 0
  for arr in model.trainable_variables:
      final_sparsity += len(tf.where(arr==0.0))
  print("Sparsity: {}".format( final_sparsity/592933))
  
  

Epoch: 1
1405/1405 [==============================] - 6s 4ms/step - loss: 0.4783 - accuracy: 0.8244 - val_loss: 0.6895 - val_accuracy: 0.7446
Sparsity: 0.9298858387035297
Epoch: 2


<ipython-input-111-41b056279a25>:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  new_weights = np.array(weights)


1405/1405 [==============================] - 6s 4ms/step - loss: 1.4223 - accuracy: 0.4125 - val_loss: 1.1138 - val_accuracy: 0.5659
Sparsity: 0.9315740564279607
Epoch: 3
1405/1405 [==============================] - 6s 4ms/step - loss: 1.1890 - accuracy: 0.5430 - val_loss: 0.9097 - val_accuracy: 0.6487
Sparsity: 0.9339571250040055
Epoch: 4
1405/1405 [==============================] - 6s 4ms/step - loss: 1.0822 - accuracy: 0.5983 - val_loss: 0.8770 - val_accuracy: 0.6602
Sparsity: 0.9358814570954898
Epoch: 5
1405/1405 [==============================] - 6s 4ms/step - loss: 1.0200 - accuracy: 0.6209 - val_loss: 0.8673 - val_accuracy: 0.6630
Sparsity: 0.9373892159822442
Epoch: 6
1405/1405 [==============================] - 6s 4ms/step - loss: 0.9847 - accuracy: 0.6323 - val_loss: 0.8554 - val_accuracy: 0.6642
Sparsity: 0.9385917127230227
Epoch: 7
1405/1405 [==============================] - 6s 4ms/step - loss: 0.9552 - accuracy: 0.6428 - val_loss: 0.8836 - val_accuracy: 0.6535
Sparsity: 0.

In [ ]:
prune_weights = model.get_weights()
prune_test_weights = np.array(prune_weights)
sparse = []
new_zero_wts, new_total_params = 0, 0
for val in range (12):
  z, t, sparsity_val = sparsity(prune_test_weights[val])
  new_zero_wts += z
  new_total_params += t
  sparse.append(sparsity_val)

final_sparsity = 0
for arr in model.trainable_variables:
    final_sparsity += len(tf.where(arr==0.0))
print("Sparsity: {}".format( final_sparsity/592933))

Sparsity: 0.968515835684639


<ipython-input-116-af0ce0c3c1e2>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  prune_test_weights = np.array(prune_weights)


In [ ]:
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 4ms/step - loss: 0.7411 - accuracy: 0.7224


In [ ]:
score = (results[1] + new_zero_wts / new_total_params) / 2
print(score)

0.8428717665285377


In [ ]:
# sparsity_val = (sum(sparse))/(len(sparse))
# accuracy_val = results[1]
# sparsity_list.append(sparsity_val)
# accuracy_list.append(accuracy_val)
# print(sparsity_list)
# print(accuracy_list)

[0.7958669427000448, 0.8036446983431592, 0.8157233202898944, 0.8237027238916467, 0.8324106863987297, 0.8405716213179225, 0.8474994070735976, 0.8537342648447295, 0.8624042616950139, 0.8701754499364781, 0.8950132876266669]
[0.742178201675415, 0.7342574000358582, 0.7390099167823792, 0.7326732873916626, 0.7370297312736511, 0.7283168435096741, 0.7287128567695618, 0.7192079424858093, 0.7112871408462524, 0.7128713130950928, 0.7100989818572998]


In [ ]:
# import pandas as pd
# sparsity_dict = {'Sparsity': sparsity_list}
# accuracy_dict = {'Accuracy': accuracy_list}

# sparsity_df = pd.DataFrame(sparsity_dict)
# accuracy_df = pd.DataFrame(accuracy_dict)

# sparsity_df.to_csv('Magnitude_Pruning_Sparsity.csv')
# accuracy_df.to_csv('Magnitude_Pruning_Accuracy.csv')

In [ ]:
 # you need to save the model's weights, naming it 'my_model_weights.h5'
model.save_weights("my_model_weights.h5")

# running this cell will immediately download a file called 'my_model_weights.h5'
from google.colab import files
files.download("my_model_weights.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Baseline Attempt
## Score: 0.53

In [ ]:
baseline_weights = np.array(original_weights)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [ ]:
# perform pruning here
mask = []
sparse = []
zero_wts, total_params = 0, 0
for val in range (12):
  baseline_weights[val][np.where(np.abs(baseline_weights[val]) < 0.0253)] = 0.0
  wts = baseline_weights[val]
  z, t, sparsity_val = sparsity(wts)
  zero_wts += z
  total_params += t
  sparse.append(sparsity_val)

final_sparsity = 0
for arr in model.trainable_variables:
    final_sparsity += len(tf.where(arr==0.0))
print("Sparsity: {}".format( final_sparsity/592933))

# you can use set_weights() to set some weights to zero, e.g.,
# some references for pruning techniques: https://arxiv.org/pdf/1810.05270v2.pdf, https://arxiv.org/pdf/2001.04062.pdf

model.set_weights(baseline_weights)


Sparsity is: 0.48413984392896103


In [ ]:
# evaluate again to see how the accuracy changes
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 4ms/step - loss: 0.8446 - accuracy: 0.6792


In [ ]:
score = (0.6792 + zero_wts / total_params) / 2
print(score)


0.5308340854700279


In [ ]:
# # you need to save the model's weights, naming it 'my_model_weights.h5'
model.save_weights("my_model_weights.h5")

# # running this cell will immediately download a file called 'my_model_weights.h5'
from google.colab import files
files.download("my_model_weights.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>